# Exercise 07 : Cross Validation (Hyperparameter Tuning)
Here we change the code of Exercise06 practice and seek best parameters (maxDepth, maxBins) for decision tree classifier.

We use ParamGridBuilder here and please compare the source code between Exercise 06 and Exercise 07.

In [2]:
# Read dataset
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, TimestampType
df = (sqlContext.read.format("csv").
  option("header", "true").
  option("nullValue", "NA").
  option("inferSchema", True).
  load("wasbs://container01@demostore01.blob.core.windows.net/flight_weather.csv"))

In [3]:
# Select required columns
df = df.select(
  "ARR_DEL15",
  "MONTH",
  "DAY_OF_WEEK",
  "UNIQUE_CARRIER",
  "ORIGIN",
  "DEST",
  "CRS_DEP_TIME",
  "CRS_ARR_TIME",
  "RelativeHumidityOrigin",
  "AltimeterOrigin",
  "DryBulbCelsiusOrigin",
  "WindSpeedOrigin",
  "VisibilityOrigin",
  "DewPointCelsiusOrigin",
  "RelativeHumidityDest",
  "AltimeterDest",
  "DryBulbCelsiusDest",
  "WindSpeedDest",
  "VisibilityDest",
  "DewPointCelsiusDest")

In [4]:
# Drop rows with null values
df = df.dropna()

In [5]:
# Convert categorical values to indexer (0, 1, ...)
from pyspark.ml.feature import StringIndexer
uniqueCarrierIndexer = StringIndexer(inputCol="UNIQUE_CARRIER", outputCol="Indexed_UNIQUE_CARRIER").fit(df)
originIndexer = StringIndexer(inputCol="ORIGIN", outputCol="Indexed_ORIGIN").fit(df)
destIndexer = StringIndexer(inputCol="DEST", outputCol="Indexed_DEST").fit(df)
arrDel15Indexer = StringIndexer(inputCol="ARR_DEL15", outputCol="Indexed_ARR_DEL15").fit(df)

In [6]:
# Assemble feature columns
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(
  inputCols = [
    "MONTH",
    "DAY_OF_WEEK",
    "Indexed_UNIQUE_CARRIER",
    "Indexed_ORIGIN",
    "Indexed_DEST",
    "CRS_DEP_TIME",
    "CRS_ARR_TIME",
    "RelativeHumidityOrigin",
    "AltimeterOrigin",
    "DryBulbCelsiusOrigin",
    "WindSpeedOrigin",
    "VisibilityOrigin",
    "DewPointCelsiusOrigin",
    "RelativeHumidityDest",
    "AltimeterDest",
    "DryBulbCelsiusDest",
    "WindSpeedDest",
    "VisibilityDest",
    "DewPointCelsiusDest"],
  outputCol = "features")

In [7]:
# Classify
from pyspark.ml.classification import DecisionTreeClassifier
classifier = DecisionTreeClassifier(featuresCol="features", labelCol="ARR_DEL15")

In [8]:
# Create pipeline
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[uniqueCarrierIndexer, originIndexer, destIndexer, arrDel15Indexer, assembler, classifier])

Note : The following takes a long time ...

In [10]:
# Run pipeline with ParamGridBuilder
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.tuning import TrainValidationSplit
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
### debugging !
# paramGrid = ParamGridBuilder() \
#   .addGrid(classifier.maxDepth, [10, 15]) \
#   .addGrid(classifier.maxBins, [251, 300]) \
#   .build()
paramGrid = ParamGridBuilder() \
 .addGrid(classifier.maxDepth, [10, 20, 30]) \
 .addGrid(classifier.maxBins, [300, 400, 500]) \
 .build()
tvs = TrainValidationSplit(
  estimator=pipeline,
  estimatorParamMaps=paramGrid,
  evaluator=MulticlassClassificationEvaluator(labelCol="ARR_DEL15", predictionCol="prediction"),
  trainRatio=0.8)
model = tvs.fit(df)

In [11]:
# View all results (accuracy) by each params
list(zip(model.validationMetrics, model.getEstimatorParamMaps()))

Out[ 12 ]: 
[(0.7703157044329688,
 {Param(parent=u'DecisionTreeClassifier_4c5aa8af501600f0c759', name='maxBins', doc='Max number of bins for discretizing continuous features. Must be >=2 and >= number of categories for any categorical feature.'): 300,
 Param(parent=u'DecisionTreeClassifier_4c5aa8af501600f0c759', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 10}),
 (0.7700237393638489,
 {Param(parent=u'DecisionTreeClassifier_4c5aa8af501600f0c759', name='maxBins', doc='Max number of bins for discretizing continuous features. Must be >=2 and >= number of categories for any categorical feature.'): 400,
 Param(parent=u'DecisionTreeClassifier_4c5aa8af501600f0c759', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 10}),
 (0.7692094426440184,
 {Param(parent=u'DecisionTreeClassifier_4c5aa8af501600f0c759', name='maxBins', doc='Max number of bins for discretizing continuous features. Must be >=2 and >= number of categories for any categorical feature.'): 500,
 Param(parent=u'DecisionTreeClassifier_4c5aa8af501600f0c759', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 10}),
 (0.7870560853907519,
 {Param(parent=u'DecisionTreeClassifier_4c5aa8af501600f0c759', name='maxBins', doc='Max number of bins for discretizing continuous features. Must be >=2 and >= number of categories for any categorical feature.'): 300,
 Param(parent=u'DecisionTreeClassifier_4c5aa8af501600f0c759', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 20}),
 (0.7868848650088807,
 {Param(parent=u'DecisionTreeClassifier_4c5aa8af501600f0c759', name='maxBins', doc='Max number of bins for discretizing continuous features. Must be >=2 and >= number of categories for any categorical feature.'): 400,
 Param(parent=u'DecisionTreeClassifier_4c5aa8af501600f0c759', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 20}),
 (0.7867180176912214,
 {Param(parent=u'DecisionTreeClassifier_4c5aa8af501600f0c759', name='maxBins', doc='Max number of bins for discretizing continuous features. Must be >=2 and >= number of categories for any categorical feature.'): 500,
 Param(parent=u'DecisionTreeClassifier_4c5aa8af501600f0c759', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 20}),
 (0.7622620771250495,
 {Param(parent=u'DecisionTreeClassifier_4c5aa8af501600f0c759', name='maxBins', doc='Max number of bins for discretizing continuous features. Must be >=2 and >= number of categories for any categorical feature.'): 300,
 Param(parent=u'DecisionTreeClassifier_4c5aa8af501600f0c759', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 30}),
 (0.7622751957261323,
 {Param(parent=u'DecisionTreeClassifier_4c5aa8af501600f0c759', name='maxBins', doc='Max number of bins for discretizing continuous features. Must be >=2 and >= number of categories for any categorical feature.'): 400,
 Param(parent=u'DecisionTreeClassifier_4c5aa8af501600f0c759', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 30}),
 (0.7626595365792725,
 {Param(parent=u'DecisionTreeClassifier_4c5aa8af501600f0c759', name='maxBins', doc='Max number of bins for discretizing continuous features. Must be >=2 and >= number of categories for any categorical feature.'): 500,
 Param(parent=u'DecisionTreeClassifier_4c5aa8af501600f0c759', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node

In [12]:
# Predict using best model
df10 = df.limit(10)
model.bestModel.transform(df10)\
  .select("ARR_DEL15", "prediction")\
  .show()

+---------+----------+
ARR_DEL15|prediction|
+---------+----------+
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 1| 0.0|
 0| 0.0|
 1| 0.0|
+---------+----------+